In [1]:
import numpy as np
import json
from itertools import chain

In [3]:
with open('output.txt', 'r') as f:
  data = json.load(f)

In [49]:
def filter_output(output, fps, time, seconds_threshold=2, score_threshold=0.09, alpha=0.7):

    total_predicted_frames = len(set(chain(*[content['frame'] for content in output.values()])))
    timeMin = time/60

    
    if alpha is None:
        if timeMin > 30:
            alpha = 0.2
        elif timeMin < 10:
            alpha = 0.6
        else:
            alpha = timeMin * 0.2 / 10

    alphaFrame = 1 - alpha
    alphaConf = alpha

    print(alpha)

    # Remove the brands that appears less than the threshold authorizes
    frames_threshold = seconds_threshold * fps
    output = {brand: content for brand, content in output.items() if len(content['frame']) > frames_threshold}

    output_dict = {}
    for brand, content in output.items():

        regularized_frame = 1 / (len(content['frame']) / total_predicted_frames)
        regularized_conf = 1 / (np.median(content['confidence']))
        score = 2 / (regularized_frame * alphaFrame + regularized_conf * alphaConf)
        max_conf = content['confidence'].index(max(content['confidence']))

        if score > score_threshold:
            output_dict[brand] = [score, (np.median(content['confidence'])), content['frame'][max_conf], content['bbox'][max_conf]]


    maxscore = max(content[0] for content in output_dict.values())
    output = {brand: content for brand, content in output_dict.items() if content[0] > (maxscore - 0.1)}

    return output

In [39]:
max({brand: content[0] for brand, content in brands.items()}.values())

1.509328381078612

In [44]:
max(content[0] for content in brands.values())

1.509328381078612

In [50]:
brands = filter_output(data, 5, seconds_threshold=1, time=19*60, alpha=0.1)
[brand for brand in brands]

0.1
{'Manscaped': [0.11132158330421818, 0.5236480236053467, 6192, [21.073402404785156, 258.95538330078125, 219.47967529296875, 316.30029296875]], 'Winamax': [1.5171042641456198, 0.7958248257637024, 57828, [394.0444030761719, 199.0662078857422, 430.9015197753906, 234.6209259033203]], 'Coca Cola': [0.1978929679680001, 0.5960767269134521, 39564, [0.7292270660400391, 58.13571548461914, 37.591896057128906, 101.61737060546875]], 'Uber Eats': [0.23253765410465954, 0.5049731135368347, 36708, [540.8789672851562, 59.06572341918945, 571.6105346679688, 72.69795989990234]]}


['Winamax']

In [31]:
brands

{'Manscaped': [0.10135737965153786,
  0.5236480236053467,
  6192,
  [21.073402404785156,
   258.95538330078125,
   219.47967529296875,
   316.30029296875]],
 'Winamax': [1.509328381078612,
  0.7958248257637024,
  57828,
  [394.0444030761719,
   199.0662078857422,
   430.9015197753906,
   234.6209259033203]],
 'Coca Cola': [0.18126375566966552,
  0.5960767269134521,
  39564,
  [0.7292270660400391,
   58.13571548461914,
   37.591896057128906,
   101.61737060546875]],
 'Uber Eats': [0.21438507619039435,
  0.5049731135368347,
  36708,
  [540.8789672851562,
   59.06572341918945,
   571.6105346679688,
   72.69795989990234]]}

In [26]:
# Division
total_predicted_frames = len(set(chain(*[content['frame'] for content in data.values()])))
{brand: (len(content['frame']) / total_predicted_frames) / (np.median(content['confidence'])) for brand, content in data.items()}

{'NordVPN': 1.2161660360956796}

In [27]:
# Multiplication
total_predicted_frames = len(set(chain(*[content['frame'] for content in data.values()])))
{brand: (len(content['frame']) / total_predicted_frames) * (np.median(content['confidence'])) for brand, content in data.items()}

{'NordVPN': 0.8222561478614807}

In [28]:
# Mean
{brand: ((len(content['frame']) / total_predicted_frames) + (np.median(content['confidence']))) / 2 for brand, content in data.items()}

{'NordVPN': 0.9111280739307404}

In [29]:
# Moyenne harmonique
total_predicted_frames = len(set(chain(*[content['frame'] for content in data.values()])))
{brand: 2 / ( (1/((len(content['frame']) / total_predicted_frames))) + (1/(np.median(content['confidence']))) ) for brand, content in data.items()}

{'NordVPN': 0.9024594581024673}

In [30]:
{brand: np.median(content['confidence']) for brand, content in data.items()}

{'NordVPN': 0.8222561478614807}

In [81]:
[brand for brand, content in data.items() if len(content['frame']) >= 1]

['Amazon',
 'Brilliant.org',
 'Crunchyroll',
 'Dollar Shave Club',
 'Logitech',
 'Manscaped',
 'audible']

['audible']

In [66]:
len(set(chain(*[content['frame'] for brand, content in data.items()])))

807

In [58]:
{brand: len(content['frame']) for brand, content in data.items()}

{'Amazon': 5,
 'Brilliant.org': 6,
 'Crunchyroll': 6,
 'Dollar Shave Club': 16,
 'GFuel': 1,
 'Genshin Impact': 3,
 'Logitech': 10,
 'Lootcrate': 4,
 'Manscaped': 7,
 'Uber Eats': 2,
 'audible': 745,
 'levlup': 4}